In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
import torch
import torch.nn as nn
from model import YoloV1

In [3]:
model = YoloV1(split_size = 7, num_boxes = 2, num_cls = 20)

In [5]:
from data_tranforms import create_transform

mytrans = create_transform()

In [6]:
from dataset import PascalDataset
dataset = PascalDataset(
    img_path='archive/images',
    label_path='archive/labels',
    csv='archive/100examples.csv',
    transformation=mytrans
)

In [7]:
loader = torch.utils.data.DataLoader(
        dataset,
        batch_size = batch,
        num_workers=2,
        pin_memory=True,
        shuffle=True,
        drop_last=False)

NameError: name 'batch' is not defined

In [ ]:
def drawBox(x, y):
    '''
    x, y for one example
    x torch tensor channel first
    
    '''
    
    img = x.permute(1,2,0)
    #plt.imshow(img)
    #print(y.shape)
    img = (img.numpy()*255).astype(np.uint8)
    #print(img)
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    boxes = cellboxes_to_boxes(y)
    boxes = nMS(boxes)
    #print(boxes)
    for box in boxes:
        
        x1, y1, x2, y2 = box_to_corners(box)
        #print([(x1, y1), (x2, y2)])
        #print(img.numpy())
        draw.rectangle([(x1, y1), (x2, y2)], outline ="red", width = 5)
    plt.imshow(img)

In [ ]:
def box_to_corners(boxes):
    #print(boxes)
    box_minx = boxes[2] - (boxes[4]/2)
    box_maxx = boxes[2] + (boxes[4]/2)
    
    box_miny = boxes[3] - (boxes[5]/2)
    box_maxy = boxes[3] + (boxes[5]/2)

    return [box_minx*448, box_miny*448, box_maxx*448, box_maxy*448]

In [ ]:
w = 10
h = 10
fig = plt.figure(figsize=(10, 10))
columns = 4
rows = 4

for x, y in loader:
    
    pred = model(x)
    
    for i in range(x.shape[0]):
    
        img = x[i].permute(1,2,0)
        #plt.imshow(img)
        #print(y.shape)
        img = (img.numpy()*255).astype(np.uint8)
        #print(img)
        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)
        boxes = cellboxes_to_boxes(pred)[i]
        boxes = nMS(boxes)
        #print(boxes)
        for box in boxes:

            x1, y1, x2, y2 = box_to_corners(box)
            #print([(x1, y1), (x2, y2)])
            #print(img.numpy())
            draw.rectangle([(x1, y1), (x2, y2)], outline ="red", width = 5)
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(img)
plt.show()

In [ ]:
seed = 123
torch.manual_seed(seed)

model = Yolo()
num_epochs = 10
batch = 16

w8_decay = 0
optimizer = torch.optim.Adam(model.parameters(), lr = 2e-5)


In [ ]:
#pred = torch.rand((1,7,7,30))
#tar = torch.rand((1,7,7,25))
lossfn = YoloLoss()
#loss.forward(pred, tar)

In [ ]:
def train():
    
    for epoch in range(num_epochs):
        
        loop = tqdm(loader, leave = True)
        
        mean_loss = []
        
        num_crct = 0
        num_sample = 0
        
        for b_id, (x, y) in enumerate(loop):

            pred = model(x)
            #print("claculated out...")
            loss = lossfn(pred, y)
            l = loss.item()
            #print("calculated loss...")
            optimizer.zero_grad()
            loss.backward()
            #print("calculated gradient...")
            optimizer.step()

            '''
            num_crct = (pred.max(1)[1] == y).sum()
            num_sample = x.shape[0]
            '''
            #acc = num_crct/num_sample
            
            mean_loss.append(l)
            loop.set_postfix(loss=l)#, acc=acc)

        print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}")


In [ ]:
train()

In [ ]:
class YoloLossAP(nn.Module):
    """
    Calculate the loss for yolo (v1) model
    """

    def __init__(self, S=7, B=2, C=20):
        super(YoloLossAP, self).__init__()
        self.mse = nn.MSELoss(reduction="sum")

        """
        S is split size of image (in paper 7),
        B is number of boxes (in paper 2),
        C is number of classes (in paper and VOC dataset is 20),
        """
        self.S = S
        self.B = B
        self.C = C

        # These are from Yolo paper, signifying how much we should
        # pay loss for no object (noobj) and the box coordinates (coord)
        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        # predictions are shaped (BATCH_SIZE, S*S(C+B*5) when inputted
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        # Calculate IoU for the two predicted bounding boxes with target bbox
        iou_b1 = iOU(predictions[..., 21:25], target[..., 21:25])
        iou_b2 = iOU(predictions[..., 26:30], target[..., 21:25])
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)

        # Take the box with highest IoU out of the two prediction
        # Note that bestbox will be indices of 0, 1 for which bbox was best
        iou_maxes, bestbox = torch.max(ious, dim=0)
        exists_box = target[..., 20].unsqueeze(3)  # in paper this is Iobj_i

        # ======================== #
        #   FOR BOX COORDINATES    #
        # ======================== #

        # Set boxes with no object in them to 0. We only take out one of the two 
        # predictions, which is the one with highest Iou calculated previously.
        box_predictions = exists_box * (
            (
                bestbox * predictions[..., 26:30]
                + (1 - bestbox) * predictions[..., 21:25]
            )
        )

        box_targets = exists_box * target[..., 21:25]

        # Take sqrt of width, height of boxes to ensure that
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )
        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2),
        )

        # ==================== #
        #   FOR OBJECT LOSS    #
        # ==================== #

        # pred_box is the confidence score for the bbox with highest IoU
        pred_box = (
            bestbox * predictions[..., 25:26] + (1 - bestbox) * predictions[..., 20:21]
        )

        object_loss = self.mse(
            torch.flatten(exists_box * pred_box),
            torch.flatten(exists_box * target[..., 20:21]),
        )

        # ======================= #
        #   FOR NO OBJECT LOSS    #
        # ======================= #

        #max_no_obj = torch.max(predictions[..., 20:21], predictions[..., 25:26])
        #no_object_loss = self.mse(
        #    torch.flatten((1 - exists_box) * max_no_obj, start_dim=1),
        #    torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        #)

        no_object_loss = self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 20:21], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1),
        )

        no_object_loss += self.mse(
            torch.flatten((1 - exists_box) * predictions[..., 25:26], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., 20:21], start_dim=1)
        )

        # ================== #
        #   FOR CLASS LOSS   #
        # ================== #

        class_loss = self.mse(
            torch.flatten(exists_box * predictions[..., :20], end_dim=-2,),
            torch.flatten(exists_box * target[..., :20], end_dim=-2,),
        )

        loss = (
            self.lambda_coord * box_loss  # first two rows in paper
            + object_loss  # third row in paper
            + self.lambda_noobj * no_object_loss  # forth row
            + class_loss  # fifth row
        )

        return loss